## CREACIÓN DE DATASETS

Vamos a trabajar con los datos de *Open Data* de ***StatsBomb*** de las 5 grandes ligas durante la temporada 2015/2016.

Para cargar los datos que nos interesan de cada partido comenzamos con la integración de información proveniente de diversas fuentes, como partidos, alineaciones y eventos, consolidándola en un dataset estructurado. Cada fila del dataset representa un partido, enriquecido con métricas calculadas que reflejan aspectos clave del rendimiento, como estadísticas agregadas de pases, disparos, recuperaciones, faltas, entre otros.

A partir de este dataset, se entrenarán modelos básicos de Machine Learning, como Random Forest, Decision Tree, Logistic Regression, K-Nearest Neighbors (KNN)... El objetivo es evaluar su capacidad de clasificación y obtener conclusiones sobre la influencia de las características seleccionadas. Este enfoque inicial proporcionará una base sólida para ampliar el análisis hacia conjuntos de datos más extensos e implementar modelos más avanzados en futuras fases del proyecto.

### Carga de datos de una competición en una temporada

#### Carga de datos (extracción y estructuración)

En primer lugar seleccionamos la competición y la temporada de esa competición que queremos estudiar.

In [1]:
from src.fetch_data import get_competition_id_and_season_id


# seleccionamos el torneo a estudiar (competición, temporada y género)
competition_name = "La Liga"
competition_gender = "male"
season_name = "2015/2016"
season_name_for_filename = "2015_2016"

competition_id, season_id = get_competition_id_and_season_id(competition_name, competition_gender, season_name)
print(f"We are going to study the tournament {competition_name + ' (' + season_name + ', ' + competition_gender + ')'} whose competition_id={competition_id} and season_id={season_id}")

We are going to study the tournament La Liga (2015/2016, male) whose competition_id=11 and season_id=27


Dada esa competición vamos a obtener cuales son todos los partidos que se han jugado en tal competición esa temporada.

In [2]:
from src.fetch_data import get_matches


matches_df = get_matches(competition_id, season_id)
print(f"Number of matches in the competition that season: {matches_df.shape[0]}")

# comprobamos que todos los partidos estén disponibles para obtener información
column_name = "match_status"
if matches_df[column_name].nunique() == 1: 
    print(f"All values in column '{column_name}' are equal: {matches_df[column_name].iloc[0]}")
else:
    print(f"The values in column '{column_name}' are different.")

Number of matches in the competition that season: 380
All values in column 'match_status' are equal: available


Tras tener todos los partidos de la competición ordenados a nuestro gusto, nos disponemos a obtener los datos relacionados con eventos, alineaciones... de cada partido. 

Estos datos son ('(x2)' se refiere tanto al equipo de casa como al de fuera y 'x' en el nombre de la variable hace referencia a 'home' y 'away'):
- Estadísticas generales:
  - Tiros:
    - Número de tiros totales (x2) --> total_shots_x
    - Ratio de tiros a puerta (x2) --> shots_on_target_ratio_x
    - Distancia promedio a la portería desde donde se hacen los tiros a puerta (x2) --> average_shot_on_target_distance_x
    - Tiros con alto xG (umbral > 0.2) (x2) --> shots_high_xG_x
    - Número de tiros dentro del área (x2) --> shots_inside_area_x
    - Ratio de tiros dentro del área (x2) --> shots_inside_area_ratio_x
    - Número de tiros con el pie (derecho e izquierdo) (x2) --> shots_foot_x
    - Número de tiros con la cabeza (x2) --> shots_head_x
    - Número de tiros con otra parte del cuerpo (x2) --> shots_other_x
  - Pases:
    - Número de pases realizados (x2) --> total_passes_x
    - Precisión de pases realizados (x2) --> pass_success_ratio_x
    - Número de pases claves realizados (x2) --> key_passes_x
    - Número de pases necesarios para hacer un gol (x2) --> passes_needed_to_make_a_shot_x
    - Número de centros al área ralizados (x2) --> cross_x
    - Precisión de los centros al área (x2) --> cross_success_ratio_x
    - Número de corners (x2) --> corners_x
  - Defensa:
    - Número de intercepciones realizadas con éxito (x2) --> interceptions_won_x
    - Número de recuperaciones/robos (x2) --> recoveries_x
    - Número de bloqueos realizados (x2) --> blocks_x
    - Número de duelos ganados (x2) --> duels_won_x
    - Número de entradas realizadas (x2) --> tackles_x
    - Ratio de entradas exitosas (x2) --> tackles_success_ratio_x
    - Número de faltas cometidas por equipo (x2) --> fouls_committed_x
    - Número de balones divididos (50-50) ganados (x2) --> 50_50_won_x
    - Número de despejes por equipo (x2) --> clearance_x
    - Número de penaltis cometidos (x2) --> penaltys_committed_x
    - Número de errores claves cometidos (x2) --> key_errors_x
    - Número de pérdidas debido a malos controles (x2) --> miscontrol_x
    - Número de tarjetas amarillas (x2) --> yellow_cards_x
    - Número de tarjetas rojas (x2) --> red_cards_x
  - Presión:
    - Número de presiones realizadas (x2) --> pressures_x
    - Número de presiones inmediatas tras pérdida (x2) --> counterpress_x
    - Presiones realizas en tercio ofensivo (x2) --> pressures_in_attacking_third_x
  - Otros:
    - Número de fueras de juego (x2) --> offside_x
    - Número de dribbles intentados (x2) --> dribbles_x
    - Ratio de dribbles exitosos (x2) --> dribbles_success_ratio_x
    - Número de cambios por lesión (x2) --> injuries_substitution_x
    - Número de jugadores lesionados que han abandonado el campo sin hacer una sustitución (x2) --> players_off_x
    - Número de pérdidas de balón (x2) --> dispossessed_x
    - Número de contragolpes/contrataques (x2) --> counterattacks_x
    - Porcentaje de posesión (x2) --> possession_percentage_x
- Estadísticas contextuales:
  - Recuperaciones:
    - Recuperaciones en tercio ofensivo (x2) --> recoveries_attacking_third_x
    - Recuperaciones en tercio medio (x2) --> recoveries_middle_third_x
    - Recuperaciones en tercio defensivo (x2) --> recoveries_defensive_third_x
  - Eventos bajo presión:
    - Tiros realizados bajo presión (x2) --> shots_under_pressure_x
    - Tiros realizados sin presión en el área (x2) --> shots_without_pressure_inside_area_x 
    - Pases realizados bajo presión (x2) --> passes_under_pressure_x
    - Pases realizados sin presión en el área (x2) --> passes_without_pressure_inside_area_x
  - Jugadas a balón parado:
    - Tiros generados desde jugadas a balón parado (x2) --> set_piece_shots_x
    - Tiros en el área generados desde jugadas a balón parado (x2) --> set_piece_shots_inside_area_x
    - Ratio de tiros a puerta generados desde jugadas a balón parado (x2) --> set_piece_shots_on_target_ratio_x
- Tácticas:
  - Número de substituciones realizadas (x2) --> substitutions_x
  - Número de substituciones debido a una razón táctica realizadas (x2) --> tactical_substitutions_x
  - Número de cambios tácticos realizados (x2) --> tactical_changes_x
  - Número de veces que se cambia la formación en el partido (x2) --> formation_changes_x
- Métricas temporales:
  - Rendimiento partidos pasados:
    - Resumen de los últimos 3 partidos (3|1|0) (x2) --> last_3_matches_form_x
    - Es válido el resumen de los últimos 3 partidos (boolean) (x2) --> is_valid_last_3_matches_form_x
    - Porcentaje de victoria en los últimos 5 partidos (x2) --> win_rate_last_5_matches_x
    - Es válido el porcentaje de victoria en los últimos 5 partidos (boolean) (x2) --> is_valid_win_rate_last_5_matches_x
    - Victoria en el último partido en casa del equipo local (boolean) --> win_last_home_match_home_team
    - Es válida la última victoria en el último partido en casa del equipo local (boolean) --> is_valid_win_last_home_match_home_team
    - Victoria en el último partido fuera del equipo visitante (boolean) --> win_last_away_match_away_team
    - Es válida la última victoria en el último partido fuera del equipo visitante (boolean) --> is_valid_win_last_away_match_away_team
  - Último partido:
    - Número de goles concedidos en el último partido (x2) --> goals_conceded_last_match_x
    - Es válido el número de goles concedidos en el último partido (boolean) (x2) --> is_valid_goals_conceded_last_match_x
    - Número de goles anotados en el último partido (x2) --> goals_scored_last_match_x
    - Es válido el número de goles anotados en el último partido (boolean) (x2) --> is_valid_goals_scored_last_match_x
  - Consistencia:
    - Desviación estándar de tiros en los últimos 3 partidos (x2) --> std_shots_last_3_matches_x
    - Es válida la desviación estándar de tiros en los últimos 3 partidos (boolean) (x2) --> is_valid_std_shots_last_3_matches_x
- Equipo ganador --> winner_team (home_team, away_team, draw)

In [3]:
from src.data_processing import process_all_matches
import pandas as pd
import os


output_dir = "data/processed/"
filename = f"{competition_name}({season_name_for_filename}_{competition_gender})_processed.csv"
output_path = os.path.join(output_dir, filename)

# comprobamos si ya hemos procesado los partidos de la competición en esa temporada
if os.path.exists(output_path):    # si ya los hemos procesado, los cargamos
    print(f"Matches from the competition already processed and saved in {output_path}")
    print("If you want to process again the matches, please delete the file.")
    matches_processed_df = pd.read_csv(output_path)
else:     # si no los hemos procesado, los procesamos y guardamos
    # procesamos todos los partido de la competición
    matches_processed_df = process_all_matches(matches_df)
    # guardamos la información procesada en un csv por si en algún momento necesitamos volver a tener que cargar esta información
    matches_processed_df.to_csv(output_path, index=False)
    print(f"All matches from the competition processed and save in {output_dir}/{filename}")

Matches from the competition already processed and saved in data/processed/La Liga(2015_2016_male)_processed.csv
If you want to process again the matches, please delete the file.


In [4]:
matches_processed_df.head(11)

,total_shots_home,total_shots_away,shots_on_target_ratio_home,shots_on_target_ratio_away,average_shots_on_target_distance_home,average_shots_on_target_distance_away,shots_high_xG_home,shots_high_xG_away,shots_inside_area_home,shots_inside_area_away,...,is_valid_goals_conceded_last_match_away,goals_scored_last_match_home,is_valid_goals_scored_last_match_home,goals_scored_last_match_away,is_valid_goals_scored_last_match_away,std_shots_last_3_matches_home,is_valid_std_shots_last_3_matches_home,std_shots_last_3_matches_away,is_valid_std_shots_last_3_matches_away,winner_team
0,5,13,0.400000,0.153846,20.462904,15.923442,0,2,3,5,...,0,0,0,0,0,0.0,0,0.0,0,home_team
1,16,10,0.312500,0.300000,25.239973,18.652679,0,2,3,6,...,0,0,0,0,0,0.0,0,0.0,0,draw
2,6,10,0.333333,0.400000,33.431447,11.773407,0,2,3,7,...,0,0,0,0,0,0.0,0,0.0,0,away_team
3,13,10,0.461538,0.400000,24.394269,9.439840,2,2,7,8,...,0,0,0,0,0,0.0,0,0.0,0,away_team
4,26,10,0.153846,0.200000,20.432657,19.849825,0,2,10,8,...,0,0,0,0,0,0.0,0,0.0,0,draw
5,6,27,0.000000,0.296296,120.000000,21.352619,0,1,4,12,...,0,0,0,0,0,0.0,0,0.0,0,draw
6,12,7,0.250000,0.000000,25.685715,120.000000,0,1,5,4,...,0,0,0,0,0,0.0,0,0.0,0,home_team
7,11,14,0.181818,0.428571,24.736200,17.236907,0,1,3,7,...,0,0,0,0,0,0.0,0,0.0,0,away_team
8,22,8,0.272727,0.500000,22.581797,18.761077,2,2,10,7,...,0,0,0,0,0,0.0,0,0.0,0,draw
9,8,10,0.375000,0.400000,33.762858,12.528144,1,3,1,7,...,0,0,0,0,0,0.0,0,0.0,0,draw


Como podemos ver ya tenemos todos los partidos de la competición en esa temporada procesados en la variable `matches_processed_df` y un archivo csv con esos datos en la carpeta `data/processed/`.

#### Procesamiento de los datos (reducción de dimensionalidad)

Vamos a reducir el número de columnas de manera que no se pierda información. Las columnas modificadas son:
- Estadísticas generales:
    - Tiros:
        - Número de tiros totales (x2) --> total_shots_x  ===>  Porcentaje de tiros totales del equipo local con respecto al total del partido --> percentage_total_shots_home
        - Tiros con alto xG (umbral > 0.2) (x2) --> shots_high_xG_x  ===>  Porcentaje de tiros con alto xG del equipo con respecto al total del partido --> percentage_shots_high_xG_home
        - Número de tiros dentro del área (x2) --> shots_inside_area_x  ===>  Porcentaje de tiros dentro del área del equipo local con respecto al total del partido --> percentage_shots_inside_area_home
        - Número de tiros con el pie (derecho e izquierdo) (x2) --> shots_foot_x  ===>  Porcentaje de tiros con el pie del equipo local con respecto al total del partido --> percentage_shots_foot_home
        - Número de tiros con la cabeza (x2) --> shots_head_x  ===>  Porcentaje de tiros con la cabeza del equipo local con respecto al total del partido --> percentage_shots_head_home
        - Número de tiros con otra parte del cuerpo (x2) --> shots_other_x  ===>  Porcentaje de tiros con otra parte del cuerpo del equipo local con respecto al total del partido --> percentage_shots_other_home
    - Pases:
        - Número de pases realizados (x2) --> total_passes_x  ===>  Porcentaje de pases del equipo local con respecto al total del partido --> percentage_total_passes_home
        - Número de pases clave realizados (x2) --> key_passes_x  ===>  Porcentaje de pases claves del equipo local con respecto al total del partido --> percentage_key_passes_home
        - Número de pases necesarios para hacer un gol (x2) --> passes_needed_to_make_a_shot_x  ===>  Diferencia de pases que necesita para hacer gol el equipo local frente al equipo visitante --> difference_passes_needed_to_make_a_shot_home
        - Número de centros al área ralizados (x2) --> cross_x  ===>  Porcentaje de centros al área realizados por el equipo local frente al total del partido --> 
        - Número de corners (x2) --> corners_x  ===>  Procentaje de corners sacados por el equipo local respecto al total del partido --> percentage_corners_home
    - Defensa:
        - Número de intercepciones realizadas con éxito (x2) --> interceptions_won_x  ===>  Procentaje del número de intercepciones realizadas con éxito del equipo local frente al total del partido --> percentage_interceptions_won_home
        - Número de recuperaciones/robos (x2) --> recoveries_x  ===>  Porcentaje de recuperaciones/robos del equipo local frente al total del partido --> percentage_recoveries_home
        - Número de bloqueos realizados (x2) --> blocks_x  ===>  Procentaje de bloqueos realizados por el equipo local respecto al total del partido --> percentage_blocks_home
        - Número de duelos ganados (x2) --> duels_won_x  ===>  Porcentaje de duelos ganados por el equipo local respecto al total del partido (x2) --> percentage_duels_won_home
        - Número de entradas realizadas (x2) --> tackles_x  ===>  Porcentaje de entradas realizadas (x2) --> percentage_tackles_home
        - Número de faltas cometidas por equipo (x2) --> fouls_committed_x  ===>  Porcentaje de faltas cometidas por el equipo local respecto al total del partido --> percentage_fouls_committed_home
        - Número de balones divididos (50-50) ganados (x2) --> 50_50_won_x  ===>  Porcentaje de balones divididos (50-50) ganados por el equipo local respecto al total del partido --> percentage_50_50_won_home
        - Número de despejes por equipo (x2) --> clearance_x  ===>  Porcentaje de despejes equipo local respecto al total del partido --> percentage_clearance_home
        - Número de penaltis cometidos (x2) --> penaltys_committed_x  ===>  Porcentaje de penaltis cometidos por el equipo local respecto al total --> percentage_penaltys_committed_home
        - Número de errores claves cometidos (x2) --> key_errors_x  ===>  Porcentaje de errores claves cometidos por el equipo local respecto al total del partido --> percentage_key_errors_home
        - Número de pérdidas debido a malos controles (x2) --> miscontrol_x  ===>  Porcentaje de pérdidas debido a malones controles del equipo local respecto al total del partido --> percentage_miscontrol_home
        - Número de tarjetas amarillas (x2) --> yellow_cards_x  ===>  Porcentaje de tarjetas amarillas del equipo local respecto al total de partido --> percentage_yellow_cards_home
        - Número de tarjetas rojas (x2) --> red_cards_x  ===> Porcentaje de tarjetas rojas del equipo local respecto al total de partido --> 
    - Presión:
        - Número de presiones realizadas (x2) --> pressures_x  ===>  Porcentaje de presiones realizadas por el equipo local respecto al total del partido --> percentage_pressures_home
        - Número de presiones inmediatas tras pérdida (x2) --> counterpress_x  ==>  Porcentaje de presiones inmediatas tras pérdida del equipo local respecto al total del partido --> percentage_counterpress_home
        - Presiones realizas en tercio ofensivo (x2) --> pressures_in_attacking_third_x  ===>  Porcentaje de presiones realizas en tercio ofensivo por el equipo local respecto al total del partido --> percentage_pressures_in_attacking_third_home
    - Otros:
        - Número de fueras de juego (x2) --> offside_x  ===>  Porcentaje de fueras de juego del equipo local respecto al total del partido --> percentage_offsides_home
        - Número de dribbles intentados (x2) --> dribbles_x  ===>  Porcentaje de dribbles intentados del equipo local respecto al total del partido --> percentage_dribbles_home
        - Número de cambios por lesión (x2) --> injuries_substitution_x  ===>  Porcentaje de cambios por lesión del equipo total respecto al total del partido --> percentage_injury_substitutions_home
        - Número de jugadores lesionados que han abandonado el campo sin hacer una sustitución (x2) --> players_off_x  ===> Porcentaje de jugadores lesionados que han abandonado el campo sin hacer una sustitución del equipo local respecto al total del partido --> percentage_players_off_home
        - Número de pérdidas de balón (x2) --> dispossessed_x  ===>  Porcentaje de pérdidas de balón del equipo local respecto al total del partido --> percentage_dispossessed_home
        - Número de contragolpes/contrataques (x2) --> counterattacks_x  ===>  Porcentaje de contragolpes/contrataques del equipo local respecto al total del partido --> percentage_counterattacks_home
- Estadística contextuales:
    - Recuperaciones:
        - Recuperaciones en tercio ofensivo (x2) --> recoveries_attacking_third_x  ===>  Porcentaje de recuperaciones en tercio ofensivo del equipo local respecto al total del partido --> percentage_recoveries_attacking_third_home
        - Recuperaciones en tercio medio (x2) --> recoveries_middle_third_x  ===>  Porcentaje de recuperaciones en tercio medio del equipo local respecto al total del partido --> percentage_recoveries_middle_third_home
        - Recuperaciones en tercio defensivo (x2) --> recoveries_defensive_third_x  ===>  Porcentaje de recuperaciones en tercio defensivo del equipo local respecto al total del partido --> percentage_recoveries_defensive_third_home
    - Eventos bajo presión:
        - Tiros realizados bajo presión (x2) --> shots_under_pressure_x  ===>  Porcentaje de tiros realizados bajo presión del equipo local respecto al total del partido --> percentage_shots_under_pressure_home
        - Tiros realizados sin presión en el área (x2) --> shots_without_pressure_inside_area_x  ===>  Porcentaje de tiros realizados sin presión en el área del equipo local respecto al total del partido --> percentage_shots_without_pressure_inside_area_home
        - Pases realizados bajo presión (x2) --> passes_under_pressure_x  ===>  Porcentaje de pases realizados bajo presión del equipo local respecto al total del partido --> percentage_passes_under_pressure_home
        - Pases realizados sin presión en el área (x2) --> passes_without_pressure_inside_area_x  ===>  Porcentaje de pases realizados sin presión en el área del equipo local respecto al total del partido --> percentage_passes_without_pressure_inside_area_x
    - Jugadas a balón parado:
        - Tiros generados desde jugadas a balón parado (x2) --> set_piece_shots_x  ===>  Porcentaje de tiros generados desde jugadas a balón parado por el equipo local respecto al total del partido --> percentage_set_piece_shots_home
        - Tiros en el área generados desde jugadas a balón parado (x2) --> set_piece_shots_inside_area_x  ===>  Porcentaje de tiros en el área generados desde jugadas a balón parado por el equipo local respecto al total del partido --> percentage_set_piece_shots_inside_area_home
- Tácticas:
    - Número de substituciones realizadas (x2) --> substitutions_x  ===>  Porcentaje de substituciones realizadas por el equipo local respecto al total --> percentage_substitutions_home
    - Número de substituciones debido a una razón táctica realizadas (x2) --> tactical_substitutions_x  ===>  Porcentaje de substituciones debido a una razón táctica realizadas por el equipo local respecto al total --> percentage_tactical_substitutions_home
    - Número de cambios tácticos realizados (x2) --> tactical_changes_x  ===>  Porcentaje de cambios tácticos realizados por el equipo local respecto al total --> percentage_tactical_changes_home
    - Número de veces que se cambia la formación en el partido (x2) --> formation_changes_x  ===>  Porcentaje de veces que cambia la formación el equipo local respecto al total --> percentage_formation_changes_home

In [5]:
from src.data_preparation import reduce_dimensionality
import pandas as pd
import os


output_dir = "data/reduced/"
filename = f"{competition_name}({season_name_for_filename}_{competition_gender})_reduced.csv"
output_path = os.path.join(output_dir, filename)

# comprobamos si ya hemos reducido el dataset de los partidos de la competición en esa temporada
if os.path.exists(output_path):    # si ya los hemos reducido antes, lo cargamos
    print(f"Matches from the competition already reduced and saved in {output_path}")
    print("If you want to reduced again the features of matches, please delete the file.")
    matches_reduced_df = pd.read_csv(output_path)
else:     # si no los hemos reducido aún, lo reducidomos y guardamos
    # reducimos la dimensaionalidad del dataset de la competición
    matches_reduced_df = reduce_dimensionality(matches_processed_df.copy())
    # guardamos la información procesada en un csv por si en algún momento necesitamos volver a tener que cargar esta información
    matches_reduced_df.to_csv(output_path, index=False)
    print(f"All matches from the competition reduced and save in {output_dir}/{filename}")

Matches from the competition already reduced and saved in data/reduced/La Liga(2015_2016_male)_reduced.csv
If you want to reduced again the features of matches, please delete the file.


In [6]:
matches_reduced_df.head(11)

,shots_on_target_ratio_home,shots_on_target_ratio_away,average_shots_on_target_distance_home,average_shots_on_target_distance_away,shots_inside_area_ratio_home,shots_inside_area_ratio_away,pass_success_ratio_home,pass_success_ratio_away,cross_success_ratio_home,cross_success_ratio_away,...,percentage_shots_under_pressure_home,percentage_shots_inside_area_under_pressure_home,percentage_passes_under_pressure_home,percentage_passes_inside_area_under_pressure_home,percentage_set_piece_shots_home,percentage_set_piece_shots_inside_area_home,percentage_substitutions_home,percentage_tactical_substitutions_home,percentage_tactical_changes_home,percentage_formation_changes_home
0,0.400000,0.153846,20.462904,15.923442,0.600000,0.384615,0.630915,0.804089,0.000000,0.000000,...,0.500000,1.000000,0.364780,0.000000,0.214286,0.285714,0.5,0.40,1.000000,0.500000
1,0.312500,0.300000,25.239973,18.652679,0.187500,0.600000,0.759191,0.690647,0.285714,0.263158,...,0.666667,0.000000,0.476190,0.333333,0.470588,0.000000,0.5,0.50,1.000000,1.000000
2,0.333333,0.400000,33.431447,11.773407,0.500000,0.700000,0.609065,0.809783,0.125000,0.222222,...,0.166667,0.250000,0.196491,0.400000,0.333333,0.250000,0.5,0.75,0.666667,0.500000
3,0.461538,0.400000,24.394269,9.439840,0.538462,0.800000,0.760377,0.676543,0.125000,0.285714,...,0.500000,0.666667,0.582857,1.000000,0.600000,0.444444,0.5,0.60,0.500000,0.500000
4,0.153846,0.200000,20.432657,19.849825,0.384615,0.800000,0.775176,0.700581,0.416667,0.272727,...,0.500000,0.000000,0.555556,0.750000,0.615385,0.384615,0.5,0.50,0.600000,0.500000
5,0.000000,0.296296,120.000000,21.352619,0.666667,0.444444,0.624642,0.832378,0.125000,0.250000,...,0.400000,0.500000,0.314516,0.200000,0.200000,0.250000,0.5,0.50,0.333333,0.500000
6,0.250000,0.000000,25.685715,120.000000,0.416667,0.571429,0.814540,0.757075,0.250000,1.000000,...,0.500000,0.500000,0.487500,1.000000,0.533333,0.333333,0.5,0.60,0.666667,0.666667
7,0.181818,0.428571,24.736200,17.236907,0.272727,0.500000,0.649518,0.864215,0.230769,0.437500,...,0.333333,0.500000,0.269565,0.142857,0.642857,0.400000,0.5,0.60,0.000000,0.500000
8,0.272727,0.500000,22.581797,18.761077,0.454545,0.875000,0.735849,0.623574,0.312500,0.375000,...,0.500000,0.500000,0.608000,0.666667,0.818182,0.692308,0.5,0.50,0.000000,0.500000
9,0.375000,0.400000,33.762858,12.528144,0.125000,0.700000,0.757895,0.686076,0.166667,0.222222,...,0.500000,0.333333,0.555556,0.500000,0.428571,0.166667,0.4,0.40,0.666667,0.500000


Una vez ya hemos reducido la dimensionalidad de los datos sin perder información y tenemos los tenemos guardados en un archivo csv en la carpeta `data/reduced/` tenemos todo preparado para poder hacer un estudio de estos datos.